In [54]:
from sklearn import datasets, cross_validation, metrics
import tensorflow as tf

import skflow
# from skflow import monitors
from scipy import misc
import os
import numpy as np
from scipy.stats import describe
import matplotlib.pyplot as plt;
%matplotlib inline

In [2]:
digits = datasets.load_digits();
digits['data'].shape # 1797x64 this things has 1797 images in 1-d vectors of size 64

digits.images.shape # 1797x8x8
digits.target.shape # labels for each image

(1797,)

### Follow similar steps to load all the images as an array of MxN where N=256*256

In [35]:
emmap = {'0': 'NE',
                      '1': 'AN', # Angry
                      '2': 'CO', # contempt
                      '3': 'DI', # disgust
                      '4': 'AF', # afraid
                      '5': 'HA', # happy
                      '6': 'SA', # sad
                      '7': 'SU', # surprised
                      '8': 'NA'  # not available
}
emotion_map = dict(zip(emmap.values(),emmap.keys()));

In [4]:
from scipy.misc import imread
inputDir = '/Users/pulsar/Documents/compneuroproj/cropped_images'
onlyfiles = [f for f in os.listdir(inputDir) if os.path.isfile(os.path.join(inputDir, f))]
print(len(onlyfiles))

1401


In [30]:
import re
def read_pgm(filename, byteorder='>'):
    """Return image data from a raw PGM file as numpy array.

    Format specification: http://netpbm.sourceforge.net/doc/pgm.html

    """
    with open(filename, 'rb') as f:
        buffer = f.read()
    try:
        header, width, height, maxval = re.search(
            b"(^P5\s(?:\s*#.*[\r\n])*"
            b"(\d+)\s(?:\s*#.*[\r\n])*"
            b"(\d+)\s(?:\s*#.*[\r\n])*"
            b"(\d+)\s(?:\s*#.*[\r\n]\s)*)", buffer).groups()
    except AttributeError:
        raise ValueError("Not a raw PGM file: '%s'" % filename)
    return np.frombuffer(buffer,
                            dtype='u1' if int(maxval) < 256 else byteorder+'u2',
                            count=int(width)*int(height),
                            offset=len(header)
                            ).reshape((int(height), int(width)))



from matplotlib import pyplot
i = 0;
images = []
for ff in onlyfiles:
    try:
        image = read_pgm(inputDir+'/'+ff)
        images.append(image);
    except ValueError:
        images.append(image);
        continue
    
image.shape
labels = []
for ff in onlyfiles:
    try:
        labels.append(emotion_map[ff[5:7]])
    except ValueError:
        print ('oops');
        break;
labels = [int(l) for l in labels];
labels = np.array([labels]).T

In [46]:
image_vectors = np.zeros((len(images), 160, 160))
for idx, im in enumerate(images):
    im_vec = np.reshape(im, [160, 160]);
    image_vectors[idx,:,:] = im_vec

In [58]:
# plt.imshow(image_vectors[0]) # 1401 images with dimensions = 160x160
labels.shape

(1401, 1)

In [82]:
X = image_vectors;
y = labels;

# Split X into test and train sets
X_train, X_test, y_train, y_test = cross_validation.train_test_split(X, y,
                                        test_size=0.2, random_state=42)

# Split X_train for validation data
X_train, X_val, y_train, y_val = cross_validation.train_test_split(X_train, 
                                    y_train, test_size=0.2,random_state=42)

In [83]:
X_train.shape

(896, 160, 160)

In [72]:
# EMBEDDING_SIZE = 20
# N_FILTERS = 10
# WINDOW_SIZE = 20
# FILTER_SHAPE1 = [WINDOW_SIZE, EMBEDDING_SIZE]
# FILTER_SHAPE2 = [WINDOW_SIZE, N_FILTERS]
# POOLING_WINDOW = 4
# POOLING_STRIDE = 2
# def conv_model(X, y):
#     X = tf.expand_dims(X, 3);
#     with tf.variable_scope('CNN_Layer1'):
#         # filtering using some number of filters
#         conv1 = skflow.ops.conv2d(X, N_FILTERS, FILTER_SHAPE1);
        
#     return skflow.models.logistic_regression(features, y)

In [74]:
classifier = skflow.TensorFlowEstimator(model_fn=conv_model, n_classes=9, 
                                        steps=500, learning_rate=0.05, batch_size=10)

classifier.fit(X_train, y_train)

/usr/local/lib/python3.5/site-packages/skflow/io/data_feeder.py:217: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  out.itemset((i, self.y[sample]), 1.0)


Step #1, avg. loss: 391.15494
Step #51, epoch #12, avg. loss: 53.59237
Step #101, epoch #25, avg. loss: 3.21669
Step #151, epoch #37, avg. loss: 1.44436
Step #201, epoch #50, avg. loss: 0.63916
Step #251, epoch #62, avg. loss: 0.24573
Step #301, epoch #75, avg. loss: 0.00523
Step #351, epoch #87, avg. loss: 0.00324
Step #401, epoch #100, avg. loss: 0.00252
Step #451, epoch #112, avg. loss: 0.00208


TensorFlowEstimator(batch_size=10, class_weight=None, continue_training=False,
          early_stopping_rounds=None, keep_checkpoint_every_n_hours=10000,
          learning_rate=0.05, max_to_keep=5,
          model_fn=<function conv_model at 0x1152d5730>, n_classes=9,
          num_cores=4, optimizer='SGD', steps=500, tf_master='',
          tf_random_seed=42, verbose=1)

In [75]:
score = metrics.accuracy_score(classifier.predict(X_test), y_test)
print('Accuracy: %f' % score)

Accuracy: 0.083333
